In [1]:
!git clone 'https://github.com/aakashvardhan/s8-normalization.git'

Cloning into 's8-normalization'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 82 (delta 34), reused 65 (delta 17), pack-reused 0
Receiving objects: 100% (82/82), 737.89 KiB | 2.92 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
import sys
sys.path.append('/content/s8-normalization')
sys.path.append('/content/s8-normalization/models')

In [3]:
from models.model import Net
from models.model_utils import model_summary
from config import get_config
config = get_config()
from utils import setup_cifar10_data
from torchvision import transforms

In [4]:
# Train Phase transformations
config['train_transforms'] = transforms.Compose([
                                      transforms.RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                                      transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                      transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomRotation(15),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                       ])

# Test Phase transformations
config['test_transforms'] = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                       ])

In [5]:
config

{'seed': 1,
 'n_channels': 32,
 'device': device(type='cuda'),
 'batch_size': 128,
 'epochs': 20,
 'lr': 0.01,
 'dropout': 0.1,
 'norm': 'bn',
 'classes': ('plane',
  'car',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'),
 'train_transforms': Compose(
     RandomAffine(degrees=[-20.0, 20.0], translate=(0.1, 0.1), scale=(0.9, 1.1))
     ColorJitter(brightness=(0.9, 1.1), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.1, 0.1))
     RandomCrop(size=(32, 32), padding=4)
     RandomHorizontalFlip(p=0.5)
     RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
     ToTensor()
     Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
 ),
 'test_transforms': Compose(
     ToTensor()
     Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
 )}

In [6]:
train_data,test_data,train_loader, test_loader = setup_cifar10_data(config)

CUDA Available? True


100%|██████████| 170498071/170498071 [00:12<00:00, 13199196.18it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
# Batch Norm
model = Net(config).to(config['device'])
model_summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
           Dropout-4           [-1, 16, 32, 32]               0
         ConvBlock-5           [-1, 16, 32, 32]               0
            Conv2d-6           [-1, 32, 32, 32]           4,608
              ReLU-7           [-1, 32, 32, 32]               0
       BatchNorm2d-8           [-1, 32, 32, 32]              64
           Dropout-9           [-1, 32, 32, 32]               0
        ConvBlock-10           [-1, 32, 32, 32]               0
           Conv2d-11           [-1, 16, 32, 32]             512
        MaxPool2d-12           [-1, 16, 16, 16]               0
  TransitionBlock-13           [-1, 16, 16, 16]               0
           Conv2d-14           [-1, 16,

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from utils import train, test

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, config['device'], train_loader, optimizer, epoch)
    test(model, config['device'], test_loader)

EPOCH: 0


Loss=1.5232387781143188 Batch_id=390 Accuracy=35.09: 100%|██████████| 391/391 [01:02<00:00,  6.25it/s]



Test set: Average loss: 1.5822, Accuracy: 4173/10000 (41.73%)

EPOCH: 1


Loss=1.4359263181686401 Batch_id=390 Accuracy=46.88: 100%|██████████| 391/391 [01:03<00:00,  6.15it/s]



Test set: Average loss: 1.3595, Accuracy: 4938/10000 (49.38%)

EPOCH: 2


...


Test set: Average loss: 0.8495, Accuracy: 6986/10000 (69.86%)

EPOCH: 18


Loss=0.8803626298904419 Batch_id=390 Accuracy=65.31: 100%|██████████| 391/391 [01:03<00:00,  6.16it/s]



Test set: Average loss: 0.8451, Accuracy: 7046/10000 (70.46%)

EPOCH: 19


Loss=1.0565707683563232 Batch_id=390 Accuracy=65.68: 100%|██████████| 391/391 [01:03<00:00,  6.16it/s]



Test set: Average loss: 0.7826, Accuracy: 7247/10000 (72.47%)

